## Work
1. 請比較 SGD optimizer 不同的 momentum 及使用 nesterov 與否的表現

In [1]:
import os
import keras

# 本作業可以不需使用 GPU, 將 GPU 設定為 "無" (若想使用可自行開啟)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95

In [7]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for lr in LEARNING_RATE:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with LR = %.6f" % (lr))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-lr-%s" % str(lr)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}


W0715 22:22:03.864975  7696 deprecation_wrapper.py:119] From C:\Users\alech\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0715 22:22:03.866928  7696 deprecation_wrapper.py:119] From C:\Users\alech\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0715 22:22:03.896845  7696 deprecation_wrapper.py:119] From C:\Users\alech\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 22:22:03.897845  7696 deprecation_wrapper.py:119] From C:\Users\alech\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 22:22:03.901833  7696 

Experiment with LR = 0.100000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


W0715 22:22:04.114360  7696 deprecation.py:323] From C:\Users\alech\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 10s 203us/step - loss: 2.1736 - acc: 0.1844 - val_loss: 2.0294 - val_acc: 0.2559
Epoch 2/50
50000/50000 [==============================] - 10s 196us/step - loss: 1.8990 - acc: 0.3020 - val_loss: 1.9646 - val_acc: 0.2870
Epoch 3/50
50000/50000 [==============================] - 9s 188us/step - loss: 1.7911 - acc: 0.3483 - val_loss: 2.0453 - val_acc: 0.3143
Epoch 4/50
50000/50000 [==============================] - 11s 215us/step - loss: 1.7766 - acc: 0.3513 - val_loss: 1.8476 - val_acc: 0.3287
Epoch 5/50
50000/50000 [==============================] - 11s 221us/step - loss: 1.7261 - acc: 0.3744 - val_loss: 1.7251 - val_acc: 0.3699
Epoch 6/50
50000/50000 [==============================] - 11s 223us/step - loss: 1.7004 - acc: 0.3846 - val_loss: 1.8128 - val_acc: 0.3479
Epoch 7/50
50000/50000 [==============================] - 11s 225us/step - loss: 1.6872 - acc: 0.3883 - val_loss: 1.7

50000/50000 [==============================] - 11s 225us/step - loss: 1.8214 - acc: 0.3486 - val_loss: 1.6404 - val_acc: 0.4224
Epoch 2/50
50000/50000 [==============================] - 12s 234us/step - loss: 1.5937 - acc: 0.4349 - val_loss: 1.5438 - val_acc: 0.4550
Epoch 3/50
50000/50000 [==============================] - 11s 222us/step - loss: 1.5023 - acc: 0.4680 - val_loss: 1.5064 - val_acc: 0.4656
Epoch 4/50
50000/50000 [==============================] - 11s 216us/step - loss: 1.4384 - acc: 0.4898 - val_loss: 1.5152 - val_acc: 0.4613
Epoch 5/50
50000/50000 [==============================] - 11s 217us/step - loss: 1.3935 - acc: 0.5055 - val_loss: 1.4922 - val_acc: 0.4698
Epoch 6/50
50000/50000 [==============================] - 11s 222us/step - loss: 1.3490 - acc: 0.5205 - val_loss: 1.4530 - val_acc: 0.4839
Epoch 7/50
50000/50000 [==============================] - 12s 236us/step - loss: 1.3126 - acc: 0.5323 - val_loss: 1.3817 - val_acc: 0.5042
Epoch 8/50
50000/50000 [==============

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()